In [3]:
#Additional Linear Modeling Topics

#Handling highly correlated variables

#Initial Linear Model of Online Spend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats import anova as sms_anova

#Load in the data 
cust_df = pd.read_csv('http://bit.ly/PMR-ch8pt1')
cust_df.head(5)
cust_df.describe(include = 'all').T

#Crete spend model
spend_m1 = smf.ols(('online_spend ~ age + credit_score + email'
                    '+ distance_to_store + online_visits'
                    '+ online_trans + store_trans + store_spend'
                    '+ sat_service + sat_selection'),
                    data = cust_df.loc[cust_df.online_spend > 0,
                                       'age':]).fit()
spend_m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           online_spend   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     2363.
Date:                Wed, 20 Aug 2025   Prob (F-statistic):               0.00
Time:                        15:35:18   Log-Likelihood:                -2157.5
No. Observations:                 418   AIC:                             4337.
Df Residuals:                     407   BIC:                             4381.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             6.7189     33.538      0.200      0.841     -59.210      72.648
email[T.yes]         -5.6893      5.807     -0.980      0.328     -17.104       5.725
age                   0.4228      0.451      0.938      0.349      -0.463       1.309
credit_score         -0.0337      0.044     -0.766      0.444      -0.120       0.053
distance_to_store    -0.0435      0.101     -0.433      0.665      -0.241       0.154
online_visits        -0.0723      0.204     -0.354      0.723      -0.473       0.329
online_trans         20.6107      0.667     30.880      0.000      19.299      21.923
store_trans           0.1350      3.212      0.042      0.966      -6.179       6.449
store_spend           0.0018      0.079      0.023      0.982      -0.153       0.157
sat_service           5.6388      3.016      1.870      0.062      -0.290      11.568
sat_selection        -4.3706      2.909     -1.502      0.134     -10.089       1.348
==============================================================================
Omnibus:                       85.767   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1424.804
Skew:                          -0.279   Prob(JB):                    4.05e-310
Kurtosis:                      12.028   Cond. No.                     1.17e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""